In [1]:
import os

os.makedirs("data/17flowers", exist_ok=True)
os.makedirs("data/102flowers", exist_ok=True)

In [ ]:
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/17flowers.tgz
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/datasplits.mat
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/trimaps.tgz

In [9]:
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/imagelabels.mat
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/setid.mat

--2023-04-19 22:57:06--  https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/octet-stream]
Saving to: ‘data/102flowers/102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  9.92MB/s    in 39s     

2023-04-19 22:57:45 (8.37 MB/s) - ‘data/102flowers/102flowers.tgz’ saved [344862509/344862509]

--2023-04-19 22:57:46--  https://thor.robots.ox.ac.uk/datasets/flowers-102/imagelabels.mat
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 502 [application/octet-stream]
Saving to: ‘data/102flowers/imagelabels.mat’

imagelabels.mat     100%[===================>]

In [ ]:
!tar -xzf data/17flowers/17flowers.tgz -C data/17flowers

In [ ]:
!tar -xzf data/102flowers/102flowers.tgz -C data/102flowers

In [ ]:
!rm data/102flowers/102flowers.tgz
!rm data/17flowers/17flowers.tgz

## Color constancy dataset using fc4

In [ ]:
import os

os.makedirs("data/cc/17flowers", exist_ok=True)
os.makedirs("data/cc/102flowers", exist_ok=True)

## 17flowers

In [7]:

import subprocess
import sys
data_path = './data/17flowers/files.txt'

f = open(data_path, 'r')
img_name = f.readlines()
f.close()

for img in img_name:
    path = './data/17flowers/jpg/'+img
    subprocess.run(['python3', 'thirdparty/fc4-python3/fc4.py', './pretrained_colorchecker/colorchecker_fold1and2.ckpt', '-1', path])

FileNotFoundError: [Errno 2] No such file or directory: './data/17flowers/files.txt'

not this one

In [4]:
!python3 ./thirdparty/fc4-python3/fc4.py test ./pretrained_colorchecker/colorchecker_fold1and2.ckpt -1 ./data/17flowers ./data/cc/17flowers

Traceback (most recent call last):
  File "/media/shiohiyoko/sandisk/IMLEX/DL/DL_CC_Classification/./thirdparty/fc4-python3/fc4.py", line 8, in <module>
    import tensorflow as tf
ModuleNotFoundError: No module named 'tensorflow'
